# Paraphrase Detection

In [381]:
from lxml import etree       
import numpy as np
import pandas as pd
import chardet
import pymorphy2
from sklearn.ensemble import GradientBoostingClassifier
import re
import cPickle as pickle
import os
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import json
import io
import scipy
import sys
import gc
import random as r
# import logging
# reload(logging)
# logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [2]:
#необходимо для красивой печати токенов на русском
def bprint(x):
    for x_ in x:
        print x_
#     print 

## The creation of dataset

In [98]:
#формирую dataframe из предложений - информация о парафразах
#Paraphrase classes: -1: non-paraphrases, 0: loose paraphrases, 1: strict paraphrases.
##Для бинарной классификации: 0 - не парафразы, 1 - парафразы
tree = etree.parse('paraphrases.xml')  
root = tree.getroot() 
corpus = root[1]
data = []

for paraphrase in corpus:
    new_pair_data = []
    for field in paraphrase:
        new_pair_data.append(field.text.encode('utf-8'))
    data.append(new_pair_data)

Paraphrases = pd.DataFrame(np.asarray(data), columns = ['pair_id', 'id_1', 'id_2', 'text_1', 'text_2', 'jaccard', 'class'])
Paraphrases[['pair_id', 'id_1', 'id_2', 'jaccard', 'class']] = \
        Paraphrases[['pair_id', 'id_1', 'id_2', 'jaccard', 'class']].apply(pd.to_numeric)
Paraphrases['class'] = Paraphrases['class'].apply(lambda x: 1 if x >=0 else 0)
Paraphrases.to_csv("Paraphrases.csv", index=False, encoding='utf-8')

In [99]:
#формирую dataframe для предложений - информация о предложениях
#Paraphrase classes: -1: non-paraphrases, 0: loose paraphrases, 1: strict paraphrases.
tree = etree.parse('corpus.xml')  
root = tree.getroot() 
corpus = root[1]
data = []

for sentense in corpus:
    new_sentense = []
    for field in sentense:
        if field.text is None:
            new_sentense.append(None)
        else:
            new_sentense.append(field.text.encode('utf-8'))
    data.append(new_sentense)

Sentences = pd.DataFrame(np.asarray(data), columns = ['id', 'text', 'agency', 'author', 'url', 'date'])
Sentences[['id']] = Sentences[['id']].apply(pd.to_numeric) 
Sentences.to_csv("Sentences.csv",index = False,  encoding='utf-8')

In [100]:
print "Train data shape: ", Paraphrases.shape
Paraphrases.head()

Train data shape:  (7227, 7)


,pair_id,id_1,id_2,text_1,text_2,jaccard,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0.650000,1
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0.500000,1
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0.611429,1
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,0.324037,0
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0.606218,1


In [101]:
print "Sentences number: ", Sentences.shape
Sentences.head()

Sentences number:  (12062, 6)


,id,text,agency,author,url,date
0,1,"Избежать ""фискального обрыва"": Сенат США подде...",РБК,None,http://top.rbc.ru/economics/01/01/2013/839229....,2013-01-01
1,2,"""Фискальный обрыв"" в США временно предотвращен.",РБК,None,http://top.rbc.ru/economics/01/01/2013/839223....,2013-01-01
2,3,Чечня попросила националистов составить кодекс...,РБК,None,http://top.rbc.ru/society/01/01/2013/839242.shtml,2013-01-01
3,4,Северокорейский лидер впервые за 19 лет поздра...,РБК,None,http://top.rbc.ru/society/01/01/2013/839227.shtml,2013-01-01
4,5,В Кот-Д`Ивуаре десятки человек погибли в давке...,РБК,None,http://top.rbc.ru/incidents/01/01/2013/839240....,2013-01-01


In [7]:
Paraphrases = pd.read_csv("Paraphrases.csv")
print Paraphrases.head()
Sentences = pd.read_csv("Sentences.csv")
print Sentences.head()

   pair_id  id_1  id_2                                             text_1  \
0        1   201  8159  Полицейским разрешат стрелять на поражение по ...   
1        2   202  8158  Право полицейских на проникновение в жилище ре...   
2        3   273  8167  Президент Египта ввел чрезвычайное положение в...   
3        4   220  8160  Вернувшихся из Сирии россиян волнует вопрос тр...   
4        5   223  8160  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                              text_2   jaccard  class  
0  Полиции могут разрешить стрелять по хулиганам ...  0.650000      1  
1  Правила внесудебного проникновения полицейских...  0.500000      1  
2  Власти Египта угрожают ввести в стране чрезвыч...  0.611429      1  
3  Самолеты МЧС вывезут россиян из разрушенной Си...  0.324037      0  
4  Самолеты МЧС вывезут россиян из разрушенной Си...  0.606218      1  
   id                                               text agency author  \
0   1  Избежать "фискального об

## Lemmatization and constructing the inverted index

In [102]:
RusLem = pymorphy2.MorphAnalyzer()
#обратный индес: {токен:список id предложений,в которых токен встречается}
inverted_index = {}
#прямой индекс: {id предложения:список токенов}
forward_index = {}

In [9]:
np_sentences = np.asarray(Sentences[['id', 'text']])

for sent in np_sentences:
    tokens = re.findall('[\w]+',sent[1].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    forward_index[sent[0]] = tokens

In [75]:
def create_inverted_index(forward_index):
    inverted_index = {}
    for sent_id in forward_index.keys():
        for token in forward_index[sent_id]:
            if token not in inverted_index:
                inverted_index[token] = [sent_id]
            elif sent_id not in inverted_index[token]:
                inverted_index[token].append(sent_id)
    return inverted_index

In [10]:
# bprint(np_sentences[:5, 1])
inverted_index = create_inverted_index(forward_index)

In [103]:
with open("./forward_index", 'wb') as res_file:
    pickle.dump(forward_index, res_file)
with open("./inverted_index", 'wb') as res_file:
    pickle.dump(inverted_index, res_file)

# Baseline model

## String - based features

In [106]:
np_paraphrases = np.asarray(Paraphrases[['pair_id', 'id_1', 'id_2']])
print np_paraphrases[len(np_paraphrases) - 10:]

[[25495 34424 34611]
 [25496 34596 34611]
 [25498 34488 34613]
 [25499 34614 34615]
 [25500 34616 34617]
 [25514 34622 34633]
 [25524 34566 34654]
 [25548 34519 34681]
 [25549 34565 34681]
 [25577 34584 34722]]


![Image](img1.png)

In [119]:
#возвращает значение признака для 1 пары предложений
#paraphrase = [id_1, id_2]
def get_string_feature(forward_index, paraphrase, n_gram = 1, gram_type = 'word', feat_num = 1):
    id_1 = paraphrase[0]
    id_2 = paraphrase[1]
    if gram_type == 'word':
        tokens_1 = forward_index[id_1]
        tokens_2 = forward_index[id_2]
        if n_gram == 1:
            set_1 = set(tokens_1)
            set_2 = set(tokens_2)
        elif n_gram == 2:
            set_1 = set([" ".join([tokens_1[idx], tokens_1[idx + 1]]) for idx in range(len(tokens_1) - 1)])
            set_2 = set([" ".join([tokens_2[idx], tokens_2[idx + 1]]) for idx in range(len(tokens_2) - 1)])
        elif n_gram == 3:
            set_1 = set([" ".join([tokens_1[idx], tokens_1[idx + 1], tokens_1[idx + 2]]) \
                                                     for idx in range(len(tokens_1) - 2)])
            set_2 = set([" ".join([tokens_2[idx], tokens_2[idx + 1], tokens_2[idx + 2]])\
                                                     for idx in range(len(tokens_2) - 2)])
        else:
            print "Not correct n_gram parameter"
            return None

    elif gram_type == 'symbol':
        text_1 = " ".join(forward_index[id_1])
        text_2 = " ".join(forward_index[id_2])
        if n_gram == 2:
            set_1 = set(["".join([text_1[idx], text_1[idx + 1]]) for idx in range(len(text_1) - 1)])
            set_2 = set(["".join([text_2[idx], text_2[idx + 1]]) for idx in range(len(text_2) - 1)])
        elif n_gram == 3:
            set_1 = set(["".join([text_1[idx], text_1[idx + 1], text_1[idx + 2]])\
                         for idx in range(len(text_1) - 2)])
            set_2 = set(["".join([text_2[idx], text_2[idx + 1], text_2[idx + 2]])\
                         for idx in range(len(text_2) - 2)])
        else:
            print "Not correct n_gram parameter"
            return None
    else:
        print "Not correct gram_type parameter"
        return None    

    if feat_num == 1:
        feature = len(set_1.intersection(set_2)) / float(len(set_1.union(set_2))) \
                                                    if len(set_1.union(set_2)) != 0 else 0
    elif feat_num == 2:
        feature = len(set_1.intersection(set_2)) / float(len(set_1)) if len(set_1) != 0 else 0
    elif feat_num == 3:
        feature = len(set_1.intersection(set_2)) / float(len(set_2)) if len(set_2) != 0 else 0
    else:
        print "Not correct feat_num parameter"
        return None
    
    return feature

#подсчет string - based features для всего датасета перифраз *создание numpy array*
#всего 15 фичей: 9 для словарных N-Gram(1, 2, 3 слов), 6 для символьных(2 и 3 символов)
def get_string_feature_for_all(forward_index, paraphrases):
    all_string_features = []
    for paraphrase in paraphrases:
#         features = [paraphrase[0]]
#         print "PAIR_ID", paraphrase[0]
#         bprint(forward_index[paraphrase[1]])
#         bprint(forward_index[paraphrase[2]])
        features = []
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=1, gram_type='word', feat_num=1))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='word', feat_num=1))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='word', feat_num=1))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=1, gram_type='word', feat_num=2))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='word', feat_num=2))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='word', feat_num=2))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=1, gram_type='word', feat_num=3))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='word', feat_num=3))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='word', feat_num=3))
        
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='symbol', feat_num=1))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='symbol', feat_num=1))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='symbol', feat_num=2))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='symbol', feat_num=2))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=2, gram_type='symbol', feat_num=3))
        features.append(get_string_feature(forward_index, paraphrase[1:], n_gram=3, gram_type='symbol', feat_num=3))
        all_string_features.append(features)
    return np.asarray(all_string_features)

In [23]:
bprint(forward_index[np_paraphrases[0, 1]])
print 
bprint(forward_index[np_paraphrases[0, 2]])
get_string_feature(forward_index, [np_paraphrases[0, 1], np_paraphrases[0, 2]], n_gram=3, gram_type='symbol', feat_num=1)

полицейский
разрешить
стрелять
на
поражение
по
гражданин
с
травматика

полиция
мочь
разрешить
стрелять
по
хулиган
с
травматика


0.4146341463414634

In [24]:
string_features = get_string_feature_for_all(forward_index, np_paraphrases)

In [25]:
print np.asarray(string_features).shape
with open("./string_features", 'wb') as res_file:
    pickle.dump(string_features, res_file)

(7227, 15)


## IR features

    N - количество предложений 
    N(w_i) - количество предложений в которых встречается терм
    avg - средняя длина предложения в датасете

![Image](./img2.png)

In [107]:
#количество предложений
N = len(forward_index.keys())

avg = 0
for sent in forward_index.keys():
    avg += len(forward_index[sent])
    
#средняя длина датасета
avg /= float(N)

In [108]:
def get_TF(token, sent_tokens):
    tf = 0
    for t in sent_tokens:
        if t == token:
            tf += 1
    return tf

def create_IDF(inverted_index):
    # IDF = {токен: его IDF}
    IDF = {}
    for token in inverted_index.keys():
        IDF[token] = np.log((N - len(inverted_index[token]) + 0.5) / float(len(inverted_index[token]) + 0.5))
    return IDF
        
def BM25(id_1, id_2, forward_index, IDF, k = 1.2, b = 0.75):
    tokens_1 = forward_index[id_1]
    tokens_2 = forward_index[id_2]
#     bprint(tokens_1)
#     print
#     bprint(tokens_2)
    bm25 = 0
    for token in tokens_1:
        tf = get_TF(token, tokens_2)
        tmp = (tf * (k + 1)) / float(tf + k * (1 - b + b * len(tokens_2)/avg))
        bm25 += IDF[token] * tmp
    return bm25

#максимальный IDF слов, которыми различаются 2 предложения
def maxIDF(id_1, id_2, forward_index, IDF):
    tokens_1 = set(forward_index[id_1])
    tokens_2 = set(forward_index[id_2])
    difference = list(tokens_1.symmetric_difference(tokens_2))
    if len(difference) == 0:
        return 0
    difference_IDF = [IDF[token] for token in difference]
    return max(difference_IDF)

#сумма IDF слов, которыми различаются 2 предложения
def sumIDF(id_1, id_2, forward_index, IDF):
    tokens_1 = set(forward_index[id_1])
    tokens_2 = set(forward_index[id_2])
    difference = list(tokens_1.symmetric_difference(tokens_2))
    difference_IDF = [IDF[token] for token in difference]
    return sum(difference_IDF)

#формирует список всех IR фичей - BM25, maxIDF, sumIDF
def get_IR_features_all(forward_index, inverted_index, paraphrases):
    all_string_features = []
    IDF = create_IDF(inverted_index)
#     print IDF
    for paraphrase in paraphrases:
        id_1 = paraphrase[1]
        id_2 = paraphrase[2]
        features = []
        features.append(BM25(id_1, id_2, forward_index, IDF))
        features.append(maxIDF(id_1, id_2, forward_index, IDF))
        features.append(sumIDF(id_1, id_2, forward_index, IDF))
        all_string_features.append(features)
    return np.asarray(all_string_features)

In [109]:
IR_features = get_IR_features_all(forward_index, inverted_index, np_paraphrases)

In [110]:
print IR_features[:5]

[[ 25.51432783   7.38237325  34.89266782]
 [ 22.21936364   8.99230873  42.4968678 ]
 [ 23.43991854   8.4814002   34.48990074]
 [ 12.27017085   8.99230873  60.84430134]
 [ 21.92113702   6.87113279  36.32048699]]


In [30]:
print avg

7.96857900846


In [111]:
#выгрузка данных
with open("./forward_index", 'r') as res_file:
    forward_index = pickle.load(res_file)
with open("./inverted_index", 'r') as res_file:
    inverted_index = pickle.load(res_file)
with open("./string_features", 'r') as res_file:
    string_features = pickle.load(res_file)

## Load Test Data

In [112]:
def get_test_data(class_cnt = 2):
    tree = etree.parse('paraphrases_gold.xml')  
    root = tree.getroot() 
    corpus = root[0]
    data = []

    for paraphrase in corpus:
        new_pair_data = []
        for field in paraphrase:
            new_pair_data.append(field.text.encode('utf-8'))
        data.append(new_pair_data)

    Paraphrases = pd.DataFrame(np.asarray(data), columns = ['pair_id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])
    Paraphrases[['pair_id', 'id_1', 'id_2', 'class']] = \
            Paraphrases[['pair_id', 'id_1', 'id_2', 'class']].apply(pd.to_numeric)
    if class_cnt == 2:
        Paraphrases['class'] = Paraphrases['class'].apply(lambda x: 1 if x >=0 else 0)
    Paraphrases.to_csv("Paraphrases_test.csv", index=False, encoding='utf-8')
    return Paraphrases

In [113]:
np_test_data = np.asarray(get_test_data())
# np_test_sentences = np_test_data[:,[1, 2]].copy()

In [114]:
#обратный индекс для тестовых данных: {токен:список id предложений,в которых токен встречается}
test_inverted_index = {}
#прямой индекс для тестовых данных: {id предложения:список токенов}
test_forward_index = {}

In [115]:
for pair in np_test_data:
    tokens = re.findall('[\w]+',pair[3].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    test_forward_index[pair[1]] = tokens
    tokens = re.findall('[\w]+',pair[4].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    test_forward_index[pair[2]] = tokens

In [120]:
test_inverted_index = create_inverted_index(test_forward_index)

In [121]:
test_embedding_index = {}
#создание прямого индекса из embedding слов
cnt = 0
for idx, word in enumerate(test_inverted_index.keys()):
    if word in WordVectors:
        test_embedding_index[word] = WordVectors[word]
    else:
        cnt += 1
print "Not in WV:", cnt
print "All words:", len(test_inverted_index)

Not in WV: 45
All words: 4239


In [122]:
with open("test_embedding_index", "w") as f:
    pickle.dump(test_embedding_index, f)

In [123]:
test_string_features = get_string_feature_for_all(test_forward_index, np_test_data[:, :3])

In [124]:
test_IR_features = get_IR_features_all(test_forward_index, test_inverted_index, np_test_data[:, :3])

In [128]:
test_mean_cosine_W2V = get_mean_embedding(np_test_data[:, :3], test_forward_index, test_embedding_index)

In [218]:
test_max_sim_W2V = get_max_similarity(np_test_data[:, :3], test_forward_index, test_embedding_index)

In [310]:
test_W2V_words_sim = get_words_similarity(np_test_data[:, :3], test_forward_index, test_embedding_index)

In [241]:
test_all_features = np.concatenate((test_string_features, test_IR_features, \
                                    test_max_sim_W2V, test_W2V_words_sim), axis = 1)

In [146]:
test_mean_cosine_W2V_expand = get_mean_embedding(np_test_data[:, :3], \
                                                 test_forward_index, test_embedding_index, need_vec=True)
test_all_features_expand = np.concatenate((test_string_features, test_IR_features, test_mean_cosine_W2V_expand), axis = 1)

In [266]:
print test_all_features.shape
print test_string_features.shape
print test_IR_features.shape
print test_mean_cosine_W2V.shape
print test_mean_cosine_W2V_expand.shape
print test_max_sim_W2V.shape

(1924, 125)
(1924, 15)
(1924, 3)
(1924, 1)
(1924, 241)
(1924, 2)


## Тrain Baseline model

### Only String Features

In [27]:
target = np.asarray(Paraphrases[['class']])

In [39]:
print string_features.shape
print target.shape

(7227, 15)
(7227, 1)


In [45]:
string_model = GradientBoostingClassifier(loss = 'deviance')

In [47]:
parametrs_grid = {'learning_rate':[0.001,0.005, 0.01, 0.05, 0.1], 'n_estimators': [100, 300, 600, 800, 1000, 1200, 1400],\
                 'subsample':[0.6, 0.7, 0.8, 1.], 'max_depth':[2, 3, 4, 5]}

In [49]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(string_model, parametrs_grid, cv = 7, verbose = 1, scoring=f1_metric, n_jobs=10)
grid_model.fit(string_features, target.ravel())

Fitting 7 folds for each of 560 candidates, totalling 3920 fits


[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  8.4min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed: 23.8min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed: 68.3min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed: 93.2min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed: 144.6min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed: 201.9min
[Parallel(n_jobs=10)]: Done 3180 tasks      | elapsed: 266.1min
[Parallel(n_jobs=10)]: Done 3920 out of 3920 | elapsed: 329.8min finished


GridSearchCV(cv=7, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'n_estimators': [100, 300, 600, 800, 1000, 1200, 1400], 'subsample': [0.6, 0.7, 0.8, 1.0], 'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1], 'max_depth': [2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=1)

In [50]:
grid_model.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.6}

In [51]:
best_GB = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=4, n_estimators=100, subsample=0.6)
best_GB.fit(string_features, target.ravel())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=0.6, verbose=0, warm_start=False)

In [61]:
test_predict = best_GB.predict(test_string_features)
print "BEST Gradient Boosting Result on String Features"
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
# print (classification_report( np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict, target_names= ['non-paraphrases', 'paraphrases']))

BEST Gradient Boosting Result on String Features
F1-score 0.789415656009
Accuracy: 0.702182952183


In [58]:
f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

0.78941565600882035

In [ ]:
with openen("string_model", "wb") as res_file:
    pickle.dump(best_GBres_files_file)

In [22]:
target = np.asarray(Paraphrases[['class']])
print IR_features.shape
print target.shape

(7227, 3)
(7227, 1)


### Only IR features

In [54]:
IR_model = GradientBoostingClassifier(loss = 'deviance')

In [62]:
parametrs_grid = {'learning_rate':[0.005, 0.01, 0.1], 'n_estimators': [100, 300, 500, 700, 1000, 1300],\
                 'subsample':[0.6, 0.8, 1.], 'max_depth':[2, 3, 4, 5]}

In [64]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(IR_model, parametrs_grid, cv = 7, verbose = 1, scoring=f1_metric, n_jobs=5)
grid_model.fit(IR_features, target.ravel())

Fitting 7 folds for each of 216 candidates, totalling 1512 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   16.6s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  3.6min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 12.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 23.1min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 38.0min
[Parallel(n_jobs=5)]: Done 1512 out of 1512 | elapsed: 48.5min finished


GridSearchCV(cv=7, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'n_estimators': [100, 300, 500, 700, 1000, 1300], 'subsample': [0.6, 0.8, 1.0], 'learning_rate': [0.005, 0.01, 0.1], 'max_depth': [2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=1)

In [65]:
grid_model.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}

In [43]:
best_GB_IR = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=3, n_estimators=100, subsample=1.0)
best_GB_IR.fit(IR_features, target.ravel())

test_predict = best_GB_IR.predict(test_IR_features)
print "BEST Gradient Boosting Result on IR Features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
# print (classification_report( np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict, target_names= ['non-paraphrases', 'paraphrases']))

BEST Gradient Boosting Result on IR Features

F1-score 0.779085872576
Accuracy: 0.668399168399


### String + IR features

In [35]:
all_features = np.concatenate((string_features, IR_features), axis = 1)
print all_features.shape

(7227, 18)


In [36]:
string_IR_model = GradientBoostingClassifier(loss = 'deviance')
parametrs_grid = {'learning_rate':[0.005, 0.01, 0.05], 'n_estimators': [100, 300, 500, 700, 1000, 1300],\
                 'subsample':[0.6, 0.7, 0.8, 1.], 'max_depth':[2, 3, 4, 5]}

In [40]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(string_IR_model, parametrs_grid, cv = 7, verbose = 1, n_jobs=5, scoring=f1_metric)
grid_model.fit(all_features, target.ravel())

Fitting 7 folds for each of 288 candidates, totalling 2016 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   33.1s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  8.6min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 27.0min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 64.1min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 97.8min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed: 140.9min
[Parallel(n_jobs=5)]: Done 2016 out of 2016 | elapsed: 170.5min finished


GridSearchCV(cv=7, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'n_estimators': [100, 300, 500, 700, 1000, 1300], 'subsample': [0.6, 0.7, 0.8, 1.0], 'learning_rate': [0.005, 0.01, 0.05], 'max_depth': [2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=1)

In [42]:
grid_model.best_params_

{'learning_rate': 0.005, 'max_depth': 2, 'n_estimators': 500, 'subsample': 0.7}

In [44]:
best_GB_IR_string = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.05, \
                                     max_depth=2, n_estimators=500, subsample=0.7)
best_GB_IR_string.fit(all_features, target.ravel())

test_predict = best_GB_IR_string.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features

F1-score 0.80221402214
Accuracy: 0.721413721414


## Experiments with Word2Vec

In [47]:
! ls -1 ../news | wc -l

291584


In [3]:
RusLem = pymorphy2.MorphAnalyzer()

In [6]:
#формирование датасета для W2V из тренировочной выборки
sentences_text = np.asarray(Sentences[['text']])
print bprint(sentences_text[0])
print len(sentences_text)
sentences = []

for sent in sentences_text:
    tokens = re.findall('[\w]+',sent[0].decode("utf-8").strip().lower(), re.U)
    tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
    sentences.append(tokens)
    
print bprint(sentences[0])
print len(sentences)

Избежать "фискального обрыва": Сенат США поддержал повышение налогов.
None
12062
избежать
фискальный
обрыв
сенат
сша
поддержать
повышение
налог
None
12062


In [8]:
#предобработка датасета новостных статей
news_files = os.listdir("../news")
# news_text = ""
with io.open("news_text", 'w', encoding='utf-8') as news_text_file:
    for idx, f in enumerate(news_files):
        if idx % 30000 == 0:
            print "Prepare", idx, 'file'
            !wc -l news_text
        with open("../news/" + f, 'r') as news_file:
            js = json.load(news_file, encoding='utf-8')
            news_text_file.write(js['text'] +  \
                                 js['title'] + '\n')

Prepare 0 file
0 news_text
Prepare 30000 file
164446 news_text
Prepare 60000 file
330209 news_text
Prepare 90000 file
496602 news_text
Prepare 120000 file
662281 news_text
Prepare 150000 file
827605 news_text
Prepare 180000 file
988928 news_text
Prepare 210000 file
1154258 news_text
Prepare 240000 file
1319022 news_text
Prepare 270000 file
1483091 news_text
В столице Ирака Багдаде в воскресенье на одном из рынков произошел взрыв, в результате которого 8 человек погибли. Об этом сообщает РИА Новости . читайте также 
Согласно имеющейся информации, взрывное устройство было спрятано в автомобиле. Сведений о числе раненных пока нет. 
Полиция ведет расследование. На место также прибыли машины скорой помощи. 
Ни одна из террористических группировок пока не взяла на себя ответственность за произошедшее.

На рынке в Багдаде прогремел взрыв: 8 человек погибли


[u'social',
 u'organizations',
 u'language',
 u'uuid',
 u'thread',
 u'author',
 u'url',
 u'ord_in_thread',
 u'title',
 u'locations',
 u'highlightText',
 u'entities',
 u'persons',
 u'external_links',
 u'text',
 u'crawled',
 u'highlightTitle',
 u'published']

In [9]:
!wc -l news_text

1603788 news_text


In [14]:
#формирование данных для обучения Word2Vec из датасета новостных статей на русском
# new_sentences = []
with open("./news_text", "r") as news_text:
    for ind, line in enumerate(news_text):
        if ind < 300400:
            continue
        if ind % 100 == 0:
#             logging.info("Prepare %d" % ind)
            sys.stderr.write("\rPrepare %d line" % ind + "\tAll Sentences:" + str(len(new_sentences)))
        for sent in line.strip().split('.'):
#             print sent
            tokens = re.findall('[\w]+',sent.decode("utf-8").strip().lower(), re.U)
            tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
            new_sentences.append(tokens)
        if len(new_sentences) > 2000000:
            break

Prepare 543300 line	All Sentences:1999447

In [17]:
bprint(new_sentences[1200])
print len(new_sentences)

фото
андрей
сенький
2000003


In [ ]:
# del(all_sentences)
del(new_sentences)

In [46]:
print len(new_sentences)

1500005


In [34]:
#объединение тренировочного датасета и датасета новостей
all_sentences = new_sentences + sentences

In [8]:
print len(sentences)

12062


In [47]:
del(model)
gc.collect()

0

In [48]:
#обучение Word2Vec
model = Word2Vec(size=120, window=5, min_count=0, iter = 5)
model.build_vocab(sentences, update=False)

In [49]:
for i in range(9):
    print i
    try:
        del(new_sentences)
    except:
        pass
    gc.collect()
    with open('news_sent_' + str(i), "r") as save_file:
        new_sentences = pickle.load(save_file)
        print len(new_sentences)
        model.build_vocab(new_sentences, update=True)
        model.train(new_sentences, total_examples=len(new_sentences), epochs=5)

0
500002
1
500001
2
500025
3
500002
4
500001
5
500002
6
500001
7
500001
8
500001


In [54]:
WordVectors = model.wv
vocab = WordVectors.vocab
print len(model.wv.vocab.keys())
print len(inverted_index.keys())

388390
10136


In [68]:
x = WordVectors.most_similar(u'страна')
for w in x:
    print w[0], w[1]

print
x = WordVectors.most_similar(u'сша')
for w in x:
    print w[0], w[1]

государство 0.709696948528
регион 0.663186311722
континент 0.594460070133
украина 0.562653005123
азербайджан 0.551063299179
россия 0.550385415554
латвия 0.533351719379
евросоюз 0.527279853821
турция 0.513354480267
америка 0.511987745762

американский 0.726210474968
вашингтон 0.697807848454
американец 0.636370003223
турция 0.618628025055
пентагон 0.600299835205
иран 0.597927153111
америка 0.592969298363
китай 0.591142654419
сирия 0.590399324894
филиппина 0.585408568382


In [85]:
# del(new_sentences)
del(RusLem)
del(sentences_text)
gc.collect()

5667

In [92]:
with open('word_vectors', "w") as f:
    pickle.dump(WordVectors, f)

In [60]:
cnt = 0
for idx, word in enumerate(inverted_index.keys()):
#     if idx % 100 == 0:
#         print idx
    if word not in vocab:
        cnt += 1
print cnt

81


In [61]:
embedding_index = {}
#создание прямого индекса из embedding слов
for idx, word in enumerate(inverted_index.keys()):
    if word in WordVectors:
        embedding_index[word] = WordVectors[word]

In [62]:
print embedding_index[u'\u0440\u0430\u0441\u0442\u0435\u043d\u0438\u0435']

[ -1.03948164e+00  -4.67318296e-01   1.66246280e-01  -2.52881908e+00
   3.44445854e-01  -2.05386305e+00  -6.67268336e-02   3.26674294e+00
  -2.14855886e+00  -5.98546386e-01  -6.41767263e-01   1.40415490e+00
   1.69666260e-02   8.70592594e-01  -1.65405655e+00  -8.84563386e-01
   1.84067285e+00   1.03923178e+00   1.37990391e+00   6.19873703e-01
  -2.81489462e-01  -1.69963147e-02   1.75444233e+00  -5.43999001e-02
   1.79508877e+00   6.65230393e-01  -7.38968730e-01   2.70580888e-01
   6.00606441e-01  -1.12671936e+00  -4.34072447e+00   3.51008654e-01
  -5.05638659e-01   1.39639014e-03   1.01712763e+00  -7.31826186e-01
  -1.21730790e-01   8.53951395e-01   1.14186680e+00   1.07774682e-01
  -2.17465115e+00  -1.13333440e+00   2.38550514e-01   1.92197824e+00
  -2.14486527e+00   1.51425824e-01   3.99049163e-01   2.66202474e+00
   4.43523079e-01  -1.30428648e+00   5.11890113e-01   2.12164545e+00
  -8.19957197e-01  -4.89127904e-01  -3.33053827e-01  -1.97383359e-01
  -1.26978791e+00  -9.82274413e-01

In [63]:
with open('word_embedding', "w") as f:
    pickle.dump(embedding_index, f)

## Word2Vec features

Возможные признаки:
    1. Усреднение векторов всех слов
    2. Максимальная близость между двумя словами в предложении - ?? 
    3. Максимальная близость между словами, которыми отличаются 2 предложения
    4. Взвешенная сумма векторов слов: умноженное на IDF слова
    3. Попарная близость все слов 2-х предложений -> N * N фичей, где N - максимальное количество слов в предложении

In [30]:
with open('word_embedding', "r") as f:
    embedding_index = pickle.load(f)

In [307]:
def get_mean_embedding(paraphrases, forward_index, embedding_index, need_vec = False):
    res_feature = []
    for pair in paraphrases:
        embeddings1 = [embedding_index[word] if word in embedding_index else np.asarray([0] * 120)\
                       for word in forward_index[pair[1]]]
        embeddings2 = [embedding_index[word] if word in embedding_index else np.asarray([0] * 120)\
                       for word in forward_index[pair[2]]]
#         print np.asarray(embeddings1).shape
        mean_1 = np.asarray(embeddings1).mean(axis = 0)
        mean_2 = np.asarray(embeddings2).mean(axis = 0)
        if not need_vec:
            res_feature.append([1 - scipy.spatial.distance.cosine(mean_1, mean_2)])
        else:
            res_feature.append(list(mean_1) + list(mean_2) + [1 -scipy.spatial.distance.cosine(mean_1, mean_2)])
    return np.asarray(res_feature)

#2 фичи: максимальная близость слов в предложении + максимальная близость слов в разнице между предложениями
def get_max_similarity(paraphrases, forward_index, embedding_index):
    res_feature = []
    for pair in paraphrases:
        embeddings1 = [embedding_index[word] if word in embedding_index else np.asarray([0] * 120)\
                       for word in forward_index[pair[1]]]
        embeddings2 = [embedding_index[word] if word in embedding_index else np.asarray([0] * 120)\
                       for word in forward_index[pair[2]]]
        
        diff = set(forward_index[pair[1]]).symmetric_difference(set(forward_index[pair[2]]))
        idxs_1 = [i for i, w in enumerate(forward_index[pair[1]]) if w in diff]
        idxs_2 = [i for i, w in enumerate(forward_index[pair[2]]) if w in diff]

        max_similarity = -1
        max_similarity_diff = -1
        for idx_1, w1 in enumerate(embeddings1):
            for idx_2, w2 in enumerate(embeddings2):
                sim = 1 - scipy.spatial.distance.cosine(w1, w2)
                if sim > max_similarity:
                    max_similarity = sim
                if idx_1 in idxs_1 and idx_2 in idxs_2:
                    if sim > max_similarity_diff:
                        max_similarity_diff = sim
        res_feature.append([max_similarity, max_similarity_diff])
        
    return np.asarray(res_feature)

def get_words_similarity(paraphrases, forward_index, embedding_index, max_words = 15):
    res_feature = []
    for pair in paraphrases:
        embeddings1 = [embedding_index[word] if word in embedding_index else np.asarray([0.000001] * 120)\
                       for word in forward_index[pair[1]]]
        embeddings2 = [embedding_index[word] if word in embedding_index else np.asarray([0.000001] * 120)\
                       for word in forward_index[pair[2]]]
        
#         bprint(forward_index[pair[1]])
#         print 
#         bprint(forward_index[pair[2]])
        feature = []
        for idx_1 in range(max_words):
            for idx_2 in range(idx_1 + 1, max_words):
                if len(embeddings1) <= idx_1 or len(embeddings2) <= idx_2:
                    feature.append(0)
                    continue
                    
#                 print scipy.spatial.distance.cosine(embeddings1[idx_1], embeddings2[idx_2])
#                 print embeddings1[idx_1]
#                 print embeddings2[idx_2]
                feature.append(1 - scipy.spatial.distance.cosine(embeddings1[idx_1], embeddings2[idx_2]))
                
        res_feature.append(feature)
    return np.asarray(res_feature)

In [178]:
Mean_W2V_feature = get_mean_embedding(np_paraphrases, forward_index, embedding_index, need_vec=False)

In [132]:
Mean_W2V_feature_expand = get_mean_embedding(np_paraphrases, forward_index, embedding_index, need_vec=True)

In [210]:
W2V_max_sim_feature = get_max_similarity(np_paraphrases, forward_index, embedding_index)

In [308]:
W2V_words_sim = get_words_similarity(np_paraphrases, forward_index, embedding_index)

In [309]:
print W2V_words_sim[9]
print W2V_words_sim.shape

[ -1.02471026e-01   1.75180030e-01   3.94775927e-02   1.24003998e-01
   2.19638087e-01   1.40358980e-01   5.50385420e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.06130852e-01   2.97192820e-02
   1.57132444e-01   1.45195588e-02   1.30185839e-01   6.78325944e-02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -7.42040506e-02
   1.52855330e-01  -7.29695145e-05   3.09203028e-02  -7.23344113e-02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   8.03789185e-02
   3.12332726e-01   1.42979779e-01   1.66312837e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   2.07902310e-01  -1.56464020e-02
   1.93784284e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [133]:
print Mean_W2V_feature.shape
print Mean_W2V_feature[15]

(7227, 1)
[ 0.68437514]


In [134]:
print Mean_W2V_feature_expand.shape
print Mean_W2V_feature_expand[10]

(7227, 241)
[ 0.11588091 -0.56805277  0.40552834 -0.58920974 -0.18696839 -0.88444752
 -0.76710904  1.06271863  0.13963798 -0.81317729 -0.35954243 -0.3871538
  1.58156538  0.05357239 -0.74392366  0.49427027  0.12111622 -0.73700917
  0.51665366 -0.33977181  0.15570831  0.00975247  0.60590458  0.12072846
  0.65226758 -0.3905386   0.51072025  0.64440018  0.83606666  0.07477744
 -1.00437212 -0.17038691  0.61684734 -0.06145687  0.34492812 -0.47568724
 -0.43611073  0.59796011 -0.35167515 -0.60352033  0.35218674  0.55607569
  0.05356154 -0.96060181  0.1617336  -0.10417278  0.20310217  0.02744379
 -0.14758456  0.5230583  -0.27601647  1.61717916  0.61725515  0.02728485
 -0.88949114 -0.27143988  0.1644586  -0.25284407 -0.03902939 -0.30906001
  0.24992552 -0.81360841  0.75376874  1.38346231 -0.92820841 -0.79101408
 -0.28832433  0.70646673  0.57049316 -0.54769182  0.7079044  -0.94162923
  0.05456032  0.00616537  0.2266406  -0.07447936  1.13989067  0.93451488
  0.75734794  0.37984902  0.20693979  0.

## String+IR+cosine_MeanW2V features

In [138]:
target = np.asarray(Paraphrases[['class']])

In [152]:
all_features = np.concatenate((string_features, IR_features, Mean_W2V_feature), axis = 1)
print all_features.shape

(7227, 19)


In [153]:
print np.asarray(string_features).shape
print np.asarray(IR_features).shape

(7227, 15)
(7227, 3)


In [154]:
Str_IR_W2V_model = GradientBoostingClassifier(loss = 'deviance')
parametrs_grid = {'learning_rate':[0.005, 0.01, 0.05], 'n_estimators': [100, 300, 500, 700, 1000, 1300],\
                 'subsample':[0.6, 0.7, 0.8, 1.], 'max_depth':[2, 3, 4, 5]}

In [ ]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(Str_IR_W2V_model, parametrs_grid, cv = 6, verbose = 1, n_jobs=5, scoring=f1_metric)
grid_model.fit(all_features, target.ravel())

Fitting 6 folds for each of 288 candidates, totalling 1728 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   32.4s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  7.5min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 29.5min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 56.1min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 96.7min
[Parallel(n_jobs=5)]: Done 1728 out of 1728 | elapsed: 140.5min finished


GridSearchCV(cv=6, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'n_estimators': [100, 300, 500, 700, 1000, 1300], 'subsample': [0.6, 0.7, 0.8, 1.0], 'learning_rate': [0.005, 0.01, 0.05], 'max_depth': [2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=1)

In [156]:
grid_model.best_params_

{'learning_rate': 0.005, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}

In [160]:
best_GB_IR_Str_W2V = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=5, n_estimators=700, subsample=0.8)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.796789492886
Accuracy: 0.710498960499


In [161]:
print best_GB_IR_Str_W2V.feature_importances_

[ 0.03389919  0.01570628  0.01230472  0.02391624  0.01903385  0.01221004
  0.0268069   0.01703021  0.01684482  0.07053947  0.11633569  0.0450963
  0.04969544  0.06067572  0.05067313  0.22149615  0.02596625  0.08398942
  0.09778017]


## String+IR+cosine_MeanW2V features+MeanEmbeddingOfSentences

In [162]:
all_features = np.concatenate((string_features, IR_features, Mean_W2V_feature_expand), axis = 1)
print all_features.shape

(7227, 259)


In [169]:
Str_IR_W2V_expand_model = GradientBoostingClassifier(loss = 'deviance')
parametrs_grid = {'learning_rate':[0.01, 0.05], 'n_estimators': [100, 300, 500, 700],\
                 'subsample':[0.6, 0.7, 0.8, 1.], 'max_depth':[2, 3, 4]}

In [ ]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(Str_IR_W2V_expand_model, parametrs_grid, cv = 6, verbose = 1, n_jobs=5, scoring=f1_metric)
grid_model.fit(all_features, target.ravel())

Fitting 6 folds for each of 96 candidates, totalling 576 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  8.6min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 111.6min


In [171]:
grid_model.best_params_

{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 300, 'subsample': 0.7}

In [172]:
best_GB_IR_Str_W2V = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=2, n_estimators=300, subsample=0.7)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features_expand)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.797238372093
Accuracy: 0.709979209979


In [173]:
print best_GB_IR_Str_W2V.feature_importances_

[  8.60579913e-03   0.00000000e+00   0.00000000e+00   0.00000000e+00
   3.79584441e-04   1.50269261e-03   0.00000000e+00   7.91349489e-05
   2.16669888e-04   9.64157408e-02   1.63191722e-01   1.27031353e-02
   8.50628635e-03   5.12466397e-02   3.34716728e-02   3.06962102e-01
   0.00000000e+00   4.70760470e-02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   6.47674241e-04   0.00000000e+00   0.00000000e+00   3.34610795e-03
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   6.04896212e-03   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.24873243e-03   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   3.40625049e-04   0.00000000e+00   4.20146167e-03   0.00000000e+00
   0.00000000e+00   7.01181852e-04   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

## String + IR + W2V_max_similatity

In [407]:
all_features = np.concatenate((string_features, IR_features, W2V_max_sim_feature), axis = 1)
print all_features.shape

test_all_features = np.concatenate((test_string_features, test_IR_features,\
                                    test_max_sim_W2V), axis = 1)
print test_all_features.shape

(7227, 20)
(1924, 20)


In [214]:
Str_IR_W2V_sim_model = GradientBoostingClassifier()
parametrs_grid = {'learning_rate':[0.01, 0.05], 'n_estimators': [100, 300, 500, 700, 1000, 1200],\
                 'subsample':[0.6, 0.7, 0.8, 1.], 'max_depth':[2, 3, 4, 5, 6]}

In [ ]:
f1_metric = make_scorer(f1_score)
grid_model = GridSearchCV(Str_IR_W2V_sim_model, parametrs_grid, cv = 6, verbose = 1, n_jobs=5, scoring=f1_metric)
grid_model.fit(all_features, target.ravel())

Fitting 6 folds for each of 240 candidates, totalling 1440 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   33.7s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  7.3min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 29.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 73.4min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 107.3min
[Parallel(n_jobs=5)]: Done 1440 out of 1440 | elapsed: 143.1min finished


GridSearchCV(cv=6, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'n_estimators': [100, 300, 500, 700, 1000, 1200], 'subsample': [0.6, 0.7, 0.8, 1.0], 'learning_rate': [0.01, 0.05], 'max_depth': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=1)

In [217]:
grid_model.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.7}

In [371]:
best_GB_IR_Str_W2V = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=3, n_estimators=900, subsample=0.7)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.804724990772
Accuracy: 0.725051975052


In [259]:
print best_GB_IR_Str_W2V.feature_importances_

[ 0.03968404  0.01541248  0.01104407  0.02452416  0.0200672   0.01319009
  0.02631749  0.01592034  0.01681601  0.07457444  0.1003791   0.04218479
  0.05145018  0.05877924  0.05359984  0.21048694  0.02186215  0.09011878
  0.00027522  0.11331345]


In [378]:
best_GB_IR_Str_W2V = RandomForestClassifier(n_estimators=2000)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.803571428571
Accuracy: 0.725571725572


In [388]:
best_GB_IR_Str_W2V = svm.LinearSVC()
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.77401894452
Accuracy: 0.652806652807


In [379]:
print best_GB_IR_Str_W2V.feature_importances_

[ 0.05158023  0.0236062   0.01348949  0.03343941  0.02100725  0.01227461
  0.04358154  0.02176164  0.01202767  0.09862912  0.11057972  0.05374822
  0.07414088  0.06618     0.08941057  0.13680704  0.02455947  0.05430478
  0.0007671   0.05810507]


## String + IR + W2V_max_similatity + WordsSimilarity

In [389]:
all_features = np.concatenate((string_features, IR_features, W2V_max_sim_feature, W2V_words_sim), axis = 1)
print all_features.shape

test_all_features = np.concatenate((test_string_features, test_IR_features,\
                                    test_max_sim_W2V, test_W2V_words_sim), axis = 1)
print test_all_features.shape

(7227, 125)
(1924, 125)


In [287]:
for i in range(np.isnan(all_features).shape[0]):
    for j in range(np.isnan(all_features).shape[1]):
        if np.isnan(all_features)[i, j] == True:
            print "sf", i, j

sf 9 34
sf 9 35
sf 9 36
sf 9 37
sf 9 38
sf 9 39
sf 26 21
sf 26 34
sf 30 24
sf 30 37
sf 30 49
sf 30 60
sf 30 70
sf 31 24
sf 31 37
sf 31 49
sf 31 60
sf 31 70
sf 38 70
sf 38 71
sf 38 72
sf 38 73
sf 38 74
sf 43 97
sf 43 98
sf 47 89
sf 47 90
sf 75 34
sf 75 35
sf 75 36
sf 75 37
sf 75 38
sf 76 70
sf 76 71
sf 86 89
sf 87 20
sf 87 21
sf 87 22
sf 87 23
sf 87 24
sf 88 89
sf 88 90
sf 88 91
sf 98 80
sf 98 81
sf 100 70
sf 100 71
sf 100 72
sf 100 73
sf 103 70
sf 113 22
sf 113 35
sf 113 47
sf 113 59
sf 113 60
sf 113 61
sf 113 62


KeyboardInterrupt: 

In [313]:
best_GB_IR_Str_W2V = GradientBoostingClassifier(loss = 'deviance', learning_rate=0.01, \
                                     max_depth=4, n_estimators=800, subsample=0.7)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.807692307692
Accuracy: 0.72972972973


In [314]:
print best_GB_IR_Str_W2V.feature_importances_

[  1.85828185e-02   6.42665313e-03   5.52733390e-03   1.31589323e-02
   9.07820943e-03   5.97622261e-03   8.58968710e-03   9.70602178e-03
   1.04071710e-02   4.30864905e-02   6.46184755e-02   2.38911618e-02
   1.99846501e-02   2.81375195e-02   1.75330051e-02   1.49365349e-01
   5.71618389e-03   3.01676291e-02   7.11480013e-10   5.59637597e-02
   1.39559767e-02   1.25586951e-02   1.42780264e-02   2.16139876e-02
   1.56586039e-02   1.89308134e-02   8.96589859e-03   8.49880295e-03
   3.95705598e-03   6.55642318e-03   4.78722360e-04   0.00000000e+00
   0.00000000e+00   0.00000000e+00   9.77746000e-03   1.47752880e-02
   1.55572489e-02   1.65616433e-02   2.00659732e-02   1.08428855e-02
   5.40351291e-03   1.86331245e-03   6.54860897e-04   2.69493884e-04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.22381241e-02
   1.15436043e-02   1.65891541e-02   1.00580569e-02   1.02203539e-02
   7.30045500e-03   3.78007762e-03   3.37342128e-03   2.87263535e-04
   0.00000000e+00   0.00000000e+00

In [390]:
best_GB_IR_Str_W2V = RandomForestClassifier(n_estimators=2000)
best_GB_IR_Str_W2V.fit(all_features, target.ravel())

test_predict = best_GB_IR_Str_W2V.predict(test_all_features)
print "BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features"
print 
print "F1-score", f1_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)
print "Accuracy:", accuracy_score(np.asarray(np_test_data[:, -1], dtype= np.int64), test_predict)

BEST Gradient Boosting Result on String+IR Features+cosine_MeanW2V features

F1-score 0.805880135695
Accuracy: 0.732328482328


In [330]:
sent1 = forward_index[np_paraphrases[13][1]]
sent2 = forward_index[np_paraphrases[13][2]]
bprint(sent1)
# bprint(r.shuffle(sent1))
print
bprint(sent2)

поздравить
год
с
год
за
19
новый
гражданин
северокорейский
лидер
впервые

лидер
кндр
впервые
за
19
год
поздравить
согражданин
с
новый
год


In [334]:
t = np.asarray(sent2)
for i in range(0, len(t) - 1, 2):
    k = t[i]
    t[i] = t[i+1]
    t[i+1] = k
    
bprint(t)

кндр
лидер
за
впервые
год
19
согражданин
поздравить
новый
с
год


In [404]:
np_paraphrases_advanced = np.concatenate((np_paraphrases, np_paraphrases * -1), axis = 0)

In [340]:
print np_paraphrases_advanced.shape
print len(forward_index)

(14454, 3)
12062


In [405]:
forward_index_advanced = forward_index.copy()
for sent_id1, sent_id2 in np_paraphrases_advanced[len(np_paraphrases):, 1:]:
    if sent_id1 < 0:
        sent = list(forward_index[sent_id1 * -1])
#         bprint(sent)
        for i in range(0, len(sent) - 1, 2):
            k = sent[i]
            sent[i] = sent[i+1]
            sent[i+1] = k
#         bprint(sent)
        forward_index_advanced[sent_id1] = sent
        
    if sent_id2 < 0:
        sent = forward_index[sent_id2 * -1]
        for i in range(0, len(sent) - 1, 2):
            k = sent[i]
            sent[i] = sent[i+1]
            sent[i+1] = k
        forward_index_advanced[sent_id2] = sent
    

In [352]:
print len(np_paraphrases)

7227


In [351]:
print np_paraphrases_advanced[0]
print np_paraphrases_advanced

[   1  201 8159]


In [406]:
print len(forward_index_advanced.keys())
bprint(forward_index_advanced[201])
print 
bprint(forward_index_advanced[-201])

24124
разрешить
полицейский
на
стрелять
по
поражение
с
гражданин
травматика

полицейский
разрешить
стрелять
на
поражение
по
гражданин
с
травматика
